In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, Subset
import tqdm
import json
import datasets
from typing import List
import os
import pandas as pd
import tiktoken
import inspect

In [2]:
!mkdir data/

mkdir: cannot create directory ‘data/’: File exists


In [3]:
encoding = tiktoken.get_encoding("gpt2")

In [4]:
encoding.n_vocab

50257

In [5]:
encoding._special_tokens

{'<|endoftext|>': 50256}

In [6]:
class Tokenizer:
    def __init__(self, tokenizer_model="gpt2"):
        gpt2_enc = tiktoken.get_encoding(tokenizer_model)
        self.enc = tiktoken.Encoding(
            name=tokenizer_model,
            pat_str=gpt2_enc._pat_str,
            mergeable_ranks=gpt2_enc._mergeable_ranks,
			special_tokens={
                **gpt2_enc._special_tokens,
                "PAD": 50257,
			},
		)
        self.tokenizer_model = tokenizer_model

        self.n_words = self.enc.n_vocab
        self.bos_id = None
        self.eos_id = self.enc.eot_token
        self.pad_id = self.enc._special_tokens["PAD"]

    def encode(self, s: str, bos: bool = False, eos: bool = False) -> List[int]:
        t = self.enc.encode(s)
        if bos and self.bos_id is not None:
            t = [self.bos_id] + t
        if eos and self.eos_id is not None:
            t = t + [self.eos_id]
        return t

    def decode(self, tokens: List[int]) -> str:
        return self.enc.decode(tokens)

In [7]:
tokenizer = Tokenizer(tokenizer_model="gpt2")

In [8]:
tokenizer.n_words

50258

In [9]:
vocab_size = 50304
batch_size = 16
block_size = 512
max_iters = 1
eval_interval = 1000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 256
n_embd = 512
n_head = 8
n_layer = 8
dropout = 0.3

target_batch_size = 8192
gradient_accumulation_steps = target_batch_size // batch_size
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95

In [10]:
gradient_accumulation_steps

512

In [11]:
torch.set_float32_matmul_precision('high')

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")

In [13]:
def encode(s): return tokenizer.encode(s, bos=False, eos=False)

def decode(l):
	try:
		return tokenizer.decode(l)
	except:
		return ""

In [14]:
ds = datasets.load_dataset("roneneldan/TinyStories")

In [15]:
ds = ds.with_format("torch")

In [16]:
ds['train'][1]

{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'}

In [17]:
# def collate_fn(batch):
#     texts = [encode(item['text'])[:block_size] for item in batch]  # Truncate to block_size
#     padded_texts = [t + [0] * (block_size - len(t)) for t in texts]  # Pad to 512
#     return {
#         'text': torch.tensor(padded_texts, dtype=torch.long)
#     }

# def collate_fn(batch):
#     texts = [encode(item['text']) for item in batch]
#     # add BOS/EOS if you like:
#     texts = [[tokenizer.bos_id] + t + [tokenizer.eos_id] for t in texts]
#     # clip or leave as is
#     maxlen = max(len(t) for t in texts)
#     padded = [t + [tokenizer.pad_id] * (maxlen - len(t)) for t in texts]
#     attention_masks = [[1]*len(t) + [0]*(maxlen-len(t)) for t in texts]
#     return {
#       'input_ids': torch.tensor(padded, dtype=torch.long),
#       'attention_mask': torch.tensor(attention_masks, dtype=torch.long),
#     }

def collate_fn(batch):
    texts = [encode(item['text'])[:block_size+1] for item in batch]  # Get one extra token for targets
    # Create inputs and targets with a 1-token shift
    batch_data = []
    for text in texts:
        if len(text) <= 1:  # Skip sequences that are too short
            continue
        
        # Use all but the last token as input
        input_text = text[:-1]
        # Use all but the first token as target (shifted by 1)
        target_text = text[1:]
        
        # Pad to block_size if needed
        if len(input_text) < block_size:
            input_text = input_text + [0] * (block_size - len(input_text))
        if len(target_text) < block_size:
            target_text = target_text + [0] * (block_size - len(target_text))
            
        batch_data.append({
            'input': torch.tensor(input_text, dtype=torch.long),
            'target': torch.tensor(target_text, dtype=torch.long)
        })
    
    # Stack all batch items
    if not batch_data:  # Handle empty batch case
        return None
    
    return {
        'input': torch.stack([item['input'] for item in batch_data]),
        'target': torch.stack([item['target'] for item in batch_data])
    }

In [18]:
eval_iters

256

In [19]:
subset_indices = list(range(eval_iters))
dataset_valid = Subset(ds['validation'], subset_indices)

In [20]:
train_dataloader = DataLoader(ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, collate_fn=collate_fn)

In [21]:
def generate_square_subsequent_mask(sz):
    """
    Generates a causal (upper-triangular) mask for a sequence of length 'sz'.
    Positions with True (or -inf when using additive masks) will be masked.
    Here, we create an additive mask with -inf for masked positions.
    """
    mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
    return mask

class Block(nn.Module):
    """Transformer block using PyTorch's MultiheadAttention with an explicit causal mask."""
    def __init__(self, n_embd, n_head):
        super().__init__()
        # PyTorch's MultiheadAttention
        self.attn = nn.MultiheadAttention(
            embed_dim=n_embd,
            num_heads=n_head,
            dropout=dropout,
            batch_first=True  # Expect input as (batch, seq, feature)
        )
        
        # Feed-forward network
        self.ffwd = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
        
        # Layer normalization layers
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        # x has shape (B, T, C)
        T = x.size(1)
        
        # Pre-LayerNorm for attention
        x_ln = self.ln1(x)
        # Create a causal mask explicitly for the current sequence length
        causal_mask = generate_square_subsequent_mask(T).to(x.device)
        
        # Self-attention: note that we pass attn_mask instead of is_causal
        attn_output, _ = self.attn(
            query=x_ln,
            key=x_ln,
            value=x_ln,
            attn_mask=causal_mask,  # Using the explicit causal mask here
            need_weights=False
        )
        x = x + attn_output
        
        # Feed-forward block with pre-LayerNorm
        x = x + self.ffwd(self.ln2(x))
        
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Token and position embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([Block(n_embd, n_head) for _ in range(n_layer)])
        
        # Final layer normalization and output projection
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)
        
		# Initialize weights for Linear and Embedding layers
        self.apply(self._init_weights)

        # Weight tying: share the weight matrix between token embeddings and the output projection
        self.token_embedding_table.weight = self.lm_head.weight

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # Obtain token embeddings and add positional embeddings
        tok_emb = self.token_embedding_table(idx)  # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        
        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x)  # (B, T, C)
            
        # Final layer normalization and output projection to logits
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        # Compute loss if targets are provided
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        Given a sequence of indices 'idx', generate 'max_new_tokens' new tokens.
        """
        for _ in range(max_new_tokens):
            # Crop the sequence to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # Get predictions
            logits, _ = self(idx_cond)
            # Focus only on the last time step
            logits = logits[:, -1, :]  # (B, vocab_size)
            # Convert logits to probabilities
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)
            # Sample from the probability distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append the new token to the sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [22]:
torch.cuda.empty_cache()

In [23]:
model = GPTLanguageModel()

# if torch.cuda.device_count() > 1:
#     model = torch.nn.DataParallel(model)

model = model.to(device)
model = torch.compile(model)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

51.237888 M parameters


In [24]:
fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and 'cuda' == str(device)
print(f"{use_fused=}")

use_fused=True


In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), eps=1e-8, fused=use_fused)

In [26]:
# T_max = len(train_dataloader)
# warmup_steps = 0.01 * T_max
# scheduler = lr_scheduler.OneCycleLR(
#     optimizer, max_lr=4e-4, total_steps=T_max, pct_start=0.01
# )
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), eps=1e-8, fused=use_fused)
true_total_steps = len(train_dataloader) // gradient_accumulation_steps
scheduler = lr_scheduler.OneCycleLR(
    optimizer, max_lr=8e-4, total_steps=true_total_steps, pct_start=0.05
)

# # Cosine Annealing Scheduler
# scheduler = lr_scheduler.CosineAnnealingLR(
#     optimizer,
# 	T_max=len(train_dataloader) // gradient_accumulation_steps,
# 	eta_min=3e-5,
# )


In [27]:
# eval_interval = len(train_dataloader) // 5
# eval_interval

In [28]:
os.makedirs("ckpt/", exist_ok=True)

In [29]:
str(device)

'cuda'

In [30]:
sample = tokenizer.decode(tokenizer.encode(ds["train"][0]["text"][:100], bos=True, eos=True))
sample

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with<|endoftext|>'

In [31]:
def generate(model, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :]  # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)  # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
    return idx

In [32]:
gradient_accumulation_steps, batch_size, target_batch_size

(512, 16, 8192)

In [33]:
with open("losses.txt", "w") as f:
	f.write("Step,Learing Rate,Training Loss,Validation Loss,Output\n")

In [34]:
for iter, batch in enumerate(tqdm.notebook.tqdm(train_dataloader, total=len(train_dataloader))):
    # inputs, targets = batch['text'], batch['text']
    inputs = batch['input'][:, :-1]
    targets = batch['input'][:, 1:]
    inputs, targets = inputs.to(device), targets.to(device)

    with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
        logits, _ = model(inputs)
        loss = F.cross_entropy(
			logits.view(-1, logits.size(-1)),
			targets.view(-1),
			ignore_index=tokenizer.pad_id
		)

    loss = loss / gradient_accumulation_steps
    loss.backward()

    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    if (iter + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    if iter % (gradient_accumulation_steps * 2) == 0 or iter == max_iters - 1:
        print(f"\nStep {iter}: Performing validation")
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")
        model.eval()
        with torch.no_grad():
            val_loss = 0
            train_loss = loss.item() * gradient_accumulation_steps
            for batch in tqdm.notebook.tqdm(valid_dataloader, total=len(valid_dataloader)):
                # inputs, targets = batch['text'], batch['text']
                inputs = batch['input'][:, :-1]
                targets = batch['input'][:, 1:]
                inputs, targets = inputs.to(device), targets.to(device)
                logits, _ = model(inputs)
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
					targets.view(-1),
					ignore_index=tokenizer.pad_id
				)
                val_loss += loss.item()

            torch.save(model.state_dict(), f"ckpt/ckpt_{iter}.pt")
            print(f"Train loss: {train_loss:.4f}")
            print(f"Validation loss: {val_loss / len(valid_dataloader):.4f}")

            prompt = "One day, a "
            prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
            output = decode(generate(model, prompt, max_new_tokens=50)[0].tolist())
            print(output)
            output = output.replace("\n", "\n")
            output = output.replace('"', "'") 
            with open("losses.txt", "a") as f:
                f.write(f"{iter},{scheduler.get_last_lr()[0]:.6f},{train_loss},{val_loss / len(valid_dataloader)},\"{output}\"\n")
        model.train()

  0%|          | 0/132483 [00:00<?, ?it/s]


Step 0: Performing validation
Learning rate: 0.000032


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 10.7571
Validation loss: 10.7184
One day, a  tap space era government Kats Beyond Desc illumination LeadRatherMom nonexistentwcs season�醒 doping administrativeFIG implementedHQ timeless flavor Blair gradientomas consolidate endowedORED Johannesprisingly cryptocurrencies campaigners Positive sorrow<|endoftext|>Asian authorised harmedcookStreamerBotisively assertingcape proportionstics BananaainingATIONS Airbusixties

Step 1024: Performing validation
Learning rate: 0.000084


  0%|          | 0/16 [00:00<?, ?it/s]

...

Train loss: 1.2178
Validation loss: 1.2902
One day, a  boy named He had a mom. Billy wasn't know, Timmy made a nest on it on. When he ran to the creek to the fairy. Timmy was confused they did something of hop.

Timmy was excited and ran outside

Step 131072: Performing validation
Learning rate: 0.000000


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 1.4570
Validation loss: 1.2902
One day, a  little mie stayed wearing a big tree.
"Try �! It looks closer!" 
The farm was a little bit close to it with it. On the sun did not want to see it away. 

Suddenly, eating quickly

Step 132096: Performing validation
Learning rate: 0.000000


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 1.4286
Validation loss: 1.2902
One day, a  Judy was something swimming in the park. Ellie was a corner of tiny, she liked to jump, but it roll around their window. Then, Ellie looked like modest car. She found her friend, "Don't normal?"

Later, Tom


In [35]:
torch.save(model.state_dict(), "final_model_tiny_stories_tiktoken_best210402025_1.pt")

In [36]:
# torch.save(model.state_dict(), "final_model.pt")

In [37]:
# # model = GPTLanguageModel()
# # model = model.to(device)
# model.load_state_dict(torch.load("/kaggle/working/ckpt/ckpt_5625.pt", weights_only=True))

# # model.eval()
# # model.to('cpu')

In [38]:
model = model.eval()

In [39]:
prompt = "There was a girl who"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

W0421 18:32:31.172000 500587 torch/_dynamo/convert_frame.py:906] [0/8] torch._dynamo hit config.cache_size_limit (8)
W0421 18:32:31.172000 500587 torch/_dynamo/convert_frame.py:906] [0/8]    function: 'forward' (/tmp/ipykernel_500587/2281288680.py:86)
W0421 18:32:31.172000 500587 torch/_dynamo/convert_frame.py:906] [0/8]    last reason: 0/0: tensor 'L['idx']' dispatch key set mismatch. expected DispatchKeySet(CUDA, BackendSelect, ADInplaceOrView, AutogradCUDA, AutocastCUDA), actual DispatchKeySet(CUDA, BackendSelect, ADInplaceOrView, AutogradCUDA)
W0421 18:32:31.172000 500587 torch/_dynamo/convert_frame.py:906] [0/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0421 18:32:31.172000 500587 torch/_dynamo/convert_frame.py:906] [0/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.


There was a girl who liked to explore the yard. One day, she saw a big dolls when it was curious because that it to get, something beautiful and shouted.

He asked Mama, "I can manage the box?"

Jari, she asked her


In [40]:
prompt = "One day, a little girl named Lily found"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

One day, a little girl named Lily found a small veil. She was a leaf and really pretty bird that she was very high on a great. 

One day, a big story is that she saw spark forward. Amy wanted her name was so pretty. It wanted to wait to


In [ ]:
# model.to('cpu')